Load kaggel Json API Token for authentication to dowanload the Dataset. Then run the below cell it will download the kaggle dataset specified and unzip it.

In [ ]:
! pip install -q kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download alexanderyyy/mnist-png
! unzip mnist-png.zip

Streaming output truncated to the last 5000 lines.
  inflating: mnist_png/train/9/1820.png  
  inflating: mnist_png/train/9/18211.png  
  inflating: mnist_png/train/9/18228.png  
  inflating: mnist_png/train/9/18241.png  
  inflating: mnist_png/train/9/18251.png  
  inflating: mnist_png/train/9/1826.png  
  inflating: mnist_png/train/9/18277.png  
  inflating: mnist_png/train/9/18296.png  
  inflating: mnist_png/train/9/183.png  
  inflating: mnist_png/train/9/18350.png  
  inflating: mnist_png/train/9/18355.png  
  inflating: mnist_png/train/9/18374.png  
  inflating: mnist_png/train/9/18377.png  
  inflating: mnist_png/train/9/18382.png  
  inflating: mnist_png/train/9/18387.png  
  inflating: mnist_png/train/9/18401.png  
  inflating: mnist_png/train/9/18405.png  
  inflating: mnist_png/train/9/18415.png  
  inflating: mnist_png/train/9/18437.png  
  inflating: mnist_png/train/9/18441.png  
  inflating: mnist_png/train/9/18448.png  
  inflating: mnist_png/train/9/18458.png  
  infla

Below cell creates a model and will be saved as model.py file. Later we use this model for training and prediction

In [ ]:
%%writefile model.py
import torch
import torch.nn as nn
import torch.nn.functional as F
class SimpleCNN(nn.Module):
    def __init__(self, dropout=0.5):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv2_drop = nn.Dropout2d(p=dropout)
        self.fc1 = nn.Linear(2304, 100) # 1600 = number channels * width * height
        self.fc2 = nn.Linear(100, 10)
        self.fc1_drop = nn.Dropout(p=dropout)

    def forward(self, x):
        x = torch.relu(F.max_pool2d(self.conv1(x), 2))
        x = torch.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))

        # flatten over channel, height and width = 1600
        x = x.view(-1, x.size(1) * x.size(2) * x.size(3))

        x = torch.relu(self.fc1_drop(self.fc1(x)))
        x = torch.softmax(self.fc2(x), dim=-1)
        return x

Writing model.py


It is divided into 3 parts


1.   If anyone wants to contribute to dataset or we want to add user drawn images to existing dataset, we just need to draw on the canvas provided and click on save button corresponding to which digit we write.This click will store the canvas to specified folder.So that entire dataset is updated.
2.   This section will load dataset inlcuding updates from user input and will train the model saved in model.py file.
It will log the training stats and displays and will save best model after each epoch.

3.   In this we again use drawable canvas to get the image to be predicted and will used saved best model to predict the canvas and prediction will be displayed.



In [ ]:
%%writefile app_dataset.py
import os
import numpy as np
import cv2
import streamlit as st
from streamlit_drawable_canvas import st_canvas

from model import SimpleCNN
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
import cv2
import time
import os
import copy
import random
import time
from PIL import Image
import pandas as pd
import PIL
plt.ion()

st.markdown("<h1 style='color: black;font-size: 2.8em;'>Create Your Dataset</h1>", unsafe_allow_html=True)

SIZE = 400
mode = st.checkbox("Draw Digit", True)
canvas_result = st_canvas(
    fill_color='#000000',
    stroke_width=20,
    stroke_color='#FFFFFF',
    background_color='#000000',
    width=SIZE,
    height=SIZE,
    drawing_mode="freedraw" if mode else "transform",
    key='canvas')
# if st.button("Save Image"):
#     if canvas_result.image_data is not None:
#         cv2.imwrite(f"test.jpg",  canvas_result.image_data)
#     else:
#         st.write("no image to save")
st.write("Specify which digit you wrote, for storing it in dataset")

col1, col2, col3,col4,col5,col6,col7,col8,col9,col10 = st.columns(10)
with col1:
    button1 = st.button('0')
with col2:
    button2 = st.button('1')
with col3:
    button3 = st.button('2')
with col4:
    button4 = st.button('3')
with col5:
    button5 = st.button('4')
with col6:
    button6 = st.button('5')
with col7:
    button7 = st.button('6')
with col8:
    button8 = st.button('7')
with col9:
    button9 = st.button('8')
with col10:
    button10 = st.button('9')
def fun():
    local_gif_path1 = "/content/1.gif"
    st.image(local_gif_path1, width=600)
a = 0
if button1:
    a += 1
    if canvas_result.image_data is not None:
        cv2.imwrite(f'/content/mnist_png/train/0/draw_{a}.png',  canvas_result.image_data)
        fun()
    else:
        st.write("no image to save")
if button2:
    a += 1

    if canvas_result.image_data is not None:
        cv2.imwrite(f'/content/mnist_png/train/1/draw_{a}.png',  canvas_result.image_data)
        fun()
    else:
        st.write("no image to save")
if button3:
    a += 1

    if canvas_result.image_data is not None:
        cv2.imwrite(f'/content/mnist_png/train/2/draw_{a}.png',  canvas_result.image_data)
        fun()
    else:
        st.write("no image to save")
if button4:
    a += 1

    if canvas_result.image_data is not None:
        cv2.imwrite(f'/content/mnist_png/train/3/draw_{a}.png',  canvas_result.image_data)
        fun()
    else:
        st.write("no image to save")
if button5:
    a += 1

    if canvas_result.image_data is not None:
        cv2.imwrite(f'/content/mnist_png/train/4/draw_{a}.png',  canvas_result.image_data)
        fun()
    else:
        st.write("no image to save")
if button6:
    a += 1

    if canvas_result.image_data is not None:
        cv2.imwrite(f'/content/mnist_png/train/5/draw_{a}.png',  canvas_result.image_data)
        fun()
    else:
        st.write("no image to save")
if button7:
    a += 1
    if canvas_result.image_data is not None:
        cv2.imwrite(f'/content/mnist_png/train/6/draw_{a}.png',  canvas_result.image_data)
        fun()
    else:
        st.write("no image to save")
if button8:
    a += 1
    if canvas_result.image_data is not None:
        cv2.imwrite(f'/content/mnist_png/train/7/draw_{a}.png',  canvas_result.image_data)
        fun()
    else:
        st.write("no image to save")
if button9:
    a += 1
    if canvas_result.image_data is not None:
        cv2.imwrite(f'/content/mnist_png/train/8/draw_{a}.png',  canvas_result.image_data)
        fun()
    else:
        st.write("no image to save")
if button10:
    a += 1
    if canvas_result.image_data is not None:
        cv2.imwrite(f'/content/mnist_png/train/9/draw_{a}.png',  canvas_result.image_data)
        fun()
    else:
        st.write("no image to save")


#####################           Part 2        ###############################################################
st.markdown("<h1 style='color: black;font-size: 2.8em;'>Train Your Model</h1>", unsafe_allow_html=True)

if st.button("Train"):
    data_transforms = {
        'train': transforms.Compose([
            transforms.Grayscale(),
            transforms.Resize(32),
            transforms.ToTensor(),

        ]),
        'test': transforms.Compose([
            transforms.Grayscale(),
            transforms.Resize(32),
            transforms.ToTensor(),
        ]),
    }
    data_dir = '/content/mnist_png'

    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                              data_transforms[x])
                      for x in ['train', 'test']}

    dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                                 shuffle=True, num_workers=2)
                  for x in ['train', 'test']}

    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
    st.write("Dataset size for training data: ", dataset_sizes['train'])
    st.write("Dataset size for validation data: ", dataset_sizes['test'])
    class_names = image_datasets['train'].classes
    st.write("Class names: ", class_names)
    local_gif_path2 = "/content/2.gif"
    st.image(local_gif_path2, width=600)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #########training####################
    model = SimpleCNN()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)


    ####
    best_accuracy = 0.0
    best_model_path = 'best_model.pth'

    ##
    # Training loop
    num_epochs = 20
    for epoch in range(num_epochs):
        for inputs, labels in dataloaders['train']:
            inputs = inputs.to(device)
            labels = labels.to(device)
            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)

            # Compute the loss
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')
        st.write(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

        # Make predictions and calculate accuracy
        correct_predictions = 0
        total_samples = 0
        with torch.no_grad():
            for inputs, labels in dataloaders['test']:
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                correct_predictions += (predicted == labels).sum().item()
                total_samples += labels.size(0)

        # Calculate accuracy
        accuracy = correct_predictions / total_samples
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            torch.save(model.state_dict(), best_model_path)
        print(f'Epoch {epoch+1}/{num_epochs}, Accuracy: {accuracy}, Best Accuracy: {best_accuracy}')
        st.write(f'Epoch {epoch+1}/{num_epochs}, Accuracy: {accuracy}, Best Accuracy: {best_accuracy}')

#####################           Part 3        ###############################################################
st.markdown("<h1 style='color: black;font-size: 2.8em;'>Write a digit between 0 to 9 for Prediction! 🖌️</h1>", unsafe_allow_html=True)

SIZE = 192
canvas_result1 = st_canvas(
    fill_color='#000000',
    stroke_width=20,
    stroke_color='#FFFFFF',
    background_color='#000000',
    width=SIZE,
    height=SIZE,
    drawing_mode="freedraw" if mode else "transform",
    key='canvas2')
if st.button("Predict"):
    if canvas_result1.image_data is not None:
        cv2.imwrite(f"test.jpg",  canvas_result1.image_data)
        model_path = 'best_model.pth'  # Path to the best model from training
        model = SimpleCNN()  # Model
        model.load_state_dict(torch.load(model_path))
        model.eval()  # Set the model to evaluation mode

        # Define transformations for inference on a single image
        transform = transforms.Compose([
                    transforms.Grayscale(),
                    transforms.Resize(32),
                    transforms.ToTensor(),

                ])

        # Load and preprocess the single image
        image_path = 'test.jpg'  # Change to the path of your single image
        image = Image.open(image_path).convert("RGB")
        input_image = transform(image).unsqueeze(0)  # Add batch dimension

        # Make prediction
        with torch.no_grad():
            output = model(input_image)
            _, predicted_class = torch.max(output, 1)

        # Display the predicted class
        print(f"The predicted class is: {predicted_class.item()}")
        st.write(f"The predicted class is: {predicted_class.item()}")
    else:
        st.write("no image to save")

Overwriting app_dataset.py


In [ ]:
!pip install -q streamlit
!npm install localtunnel -q
!pip install streamlit-drawable-canvas -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00
npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.196s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.2 MB/s eta 0:00:00


In [ ]:
!streamlit run app_dataset.py & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

35.245.71.121



  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.245.71.121:8501

npx: installed 22 in 2.54s
your url is: https://rich-friends-act.loca.lt
The predicted class is: 2
The predicted class is: 7
Epoch 1/20, Loss: 1.4611501693725586
Epoch 1/20, Accuracy: 0.9253, Best Accuracy: 0.9253
Epoch 2/20, Loss: 1.461151123046875
The predicted class is: 2
The predicted class is: 4
The predicted class is: 5
The predicted class is: 7
The predicted class is: 9
Epoch 1/20, Loss: 1.4611506462097168
Epoch 1/20, Accuracy: 0.9299, Best Accuracy: 0.9299
Epoch 2/20, Loss: 1.4613487720489502
  Stopping...


In [ ]:
#THE END